In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional,Dropout
from sklearn.metrics import mean_squared_error

# Normalize the data
scaler = MinMaxScaler()
quantity_scaled = scaler.fit_transform(df_weekly[["Quantity"]])

# Prepare data for LSTM
X = []
y = []
seq_length = 10  # Adjust sequence length as needed

for i in range(len(quantity_scaled) - seq_length):
    X.append(quantity_scaled[i:i+seq_length])
    y.append(quantity_scaled[i+seq_length])

X = np.array(X)
y = np.array(y)

# Split data into train and test sets
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build LSTM model
model = Sequential([
    LSTM(70, activation='relu', input_shape=(seq_length, 1)),
    Dense(256,activation='relu'),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=1, verbose=2)


train_forecast_scaled = model.predict(X_train)
train_forecast = scaler.inverse_transform(train_forecast_scaled)

# Plot the original training data and the forecast
plt.figure(figsize=(10, 6))
plt.plot(df_weekly.index[seq_length:seq_length+len(train_forecast)], df_weekly["Quantity"][seq_length:seq_length+len(train_forecast)], label='Original Data')
plt.plot(df_weekly.index[seq_length:seq_length+len(train_forecast)], train_forecast, color='red', label='Forecast')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Quantity')
plt.title('LSTM Model Forecasting for Quantity (Training Data)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Calculate RMSE on the training data
mse = mean_squared_error(df_weekly["Quantity"][seq_length:seq_length+len(train_forecast)], train_forecast)
print(f'Root Mean Squared Error (MSE) on Training Data: {mse}')